**Setup (for Windows 10)**

1. Create a Python 3.7 'pytorch' environment with Anaconda Navigator
2. Activate the environment and run 'install pytorch torchvision - c pytorch': https://medium.com/@iamHarin17/how-to-setup-a-python-environment-for-deep-learning-with-anaconda-f65ab78a362
3. Run 'pip install' for 'matplotlib', 'tqdm' and 'scipy'
4. Install Jupyter Notebook from Anaconda Navigator and open this notebook.

**Helpful Links**

DOS Nvidia GPU usage: https://wiki.tuflow.com/index.php?title=DOS_GPU_Usage

In [1]:
# Set global variables to run notebook

# data root
data_root = 'C:\_data/' # set to whever the data is being stored

# BrokenPipeError on Windows 10
# https://discuss.pytorch.org/t/brokenpipeerror-errno-32-broken-pipe-when-i-run-cifar10-tutorial-py/6224/3
global_num_workers = 0 # originally set to 4, so perhaps should be 4 on Mac

In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from tqdm import tqdm
from scipy import stats

In [3]:
device = torch.device('cuda:0')

In [4]:
# data_transforms = {
#     'train': transforms.Compose([
# #         transforms.Resize((310, 310)),
#         transforms.Resize((224, 224)),
# #         transforms.CenterCrop(224),
# #         transforms.RandomResizedCrop(224),
# #         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
# #         transforms.Resize((310, 310)),
#         transforms.Resize((224, 224)),
# #         transforms.Resize(256),
# #         transforms.CenterCrop(224),
# #         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [5]:
class ImageDataset(Dataset):
    def __init__(self, paths, labels, transform=None, stride = 1, max_size=None):
        self.paths = paths[::stride]
        if max_size is not None:
            self.paths = self.paths[:max_size]
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_name = self.paths[idx]
        img = Image.open(img_name).convert('RGB')
        
        img_tensor = self.transform(img)

        return img_tensor, self.labels[idx]

In [6]:
train_file = data_root + 'commercials/data/train.txt'
val_file   = data_root + 'commercials/data/val.txt'
test_file  = data_root + 'commercials/data/test.txt'

In [7]:
def read_file(filename):
    paths = []
    labels = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            video, image, label = line.split(' ')
            paths.append(data_root + 'commercials/images/{}/{:04d}.jpg'.format(video, int(image)))
            labels.append(int(label))
            
    return paths, labels

In [8]:
train_paths, Y_train = read_file(train_file)
val_paths, Y_val = read_file(val_file)
test_paths, Y_test = read_file(test_file)

In [9]:
image_datasets = {
    'train': ImageDataset(train_paths, Y_train, transform = data_transforms['train'],
                          stride=1, max_size=None),
    'val_train': ImageDataset(val_paths, Y_val, transform = data_transforms['train'],
                       stride=1, max_size=None),
    'val': ImageDataset(val_paths, Y_val, transform = data_transforms['val'],
                       stride=1, max_size=None),
    'test': ImageDataset(test_paths, Y_test, transform = data_transforms['val'],
                        stride=1, max_size=None)
}

In [10]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size = 4,
                                              shuffle = 'train' in x,
                                              num_workers = global_num_workers,
                                              pin_memory = True)
                                              for x in image_datasets}

In [11]:
dataset_sizes = {
    x: len(image_datasets[x])
    for x in image_datasets
}

In [12]:
dataset_sizes

{'train': 64130, 'val_train': 9479, 'val': 9479, 'test': 7496}

In [13]:
def safe_divide(a, b):
    return a / b if b > 0 else 0

In [14]:
def train_model(model, criterion, optimizer, scheduler, train_dl, val_dl, test_dl=None,
                num_epochs=25, return_best=False, verbose=True, log_file=None):
    print(train_dl, val_dl, test_dl)
    since = time.time() # timestamp for beginning of training

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    best_test_acc = 0.0
    best_f1 = 0.0
    best_precision = 0.0
    best_recall = 0.0
    
    phases = ['train', 'val', 'test'] if test_dl is not None else ['train', 'val']

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
                dl = dataloaders[train_dl]
                dataset_size = dataset_sizes[train_dl]
                
            elif phase == 'val':
                model.eval()   # Set model to evaluate mode
                dl = dataloaders[val_dl]
                dataset_size = dataset_sizes[val_dl]
            else:
                model.eval()
                dl = dataloaders[test_dl]
                dataset_size = dataset_sizes[test_dl]

            running_loss = 0.0
            running_corrects = 0
            true_positives = 0.
            true_negatives = 0.
            false_positives = 0.
            false_negatives = 0.

            # Iterate over data.
            for inputs, labels in dl:
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.where(
                        outputs >= 0.,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                    target = torch.where(
                        labels >= 0.5,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device)
                    )
                    loss = criterion(outputs.view(target.shape), target)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                label_vals = torch.where(
                    labels >= 0.5,
                    torch.tensor([1.]).to(device),
                    torch.tensor([0.]).to(device)
                )
                correct = preds.view(label_vals.shape) == label_vals.data
                running_corrects += torch.sum(correct)
                
                true_positives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                true_negatives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_positives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_negatives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
            
            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size
            epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
            epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
            epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

            if verbose:
                print('{} Loss: {:.4f} Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc, epoch_pre, epoch_recall, epoch_f1))
                print('TP: {} TN: {} FP: {} FN: {}'.format(
                    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))
            if log_file is not None:
                log_file.write('Phase: {0}\t'
                               'Epoch: [{1}/{2}]\t'
                               'Loss: {loss_c:.4f}\t'
                               'Acc: {acc:.4f}\t'
                               'Pre: {pre:.4f}\t'
                               'Rec: {rec:.4f}\t'
                               'F1: {f1:.4f}\t'
                               'TP: {tp} '
                               'TN: {tn} '
                               'FP: {fp} '
                               'FN: {fn}\n'.format(
                                   phase, epoch + 1, num_epochs, loss_c=epoch_loss,
                                   acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                                   f1=epoch_f1, tp=int(true_positives.data), tn=int(true_negatives.data),
                                   fp=int(false_positives.data), fn=int(false_negatives.data)
                               ))
                log_file.flush()

            # deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_acc = epoch_acc
                best_f1 = epoch_f1
                best_precision = epoch_pre
                best_recall = epoch_recall
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test' and best_epoch == epoch:
                best_test_acc = epoch_acc

        time_elapsed = time.time() - since
        print('Training time {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print()

    # Print total training time
    time_elapsed = time.time() - since
    print('\nTraining complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    if return_best:
        print('Best epoch: {}'.format(best_epoch))
        print('Best val Acc: {:4f}'.format(best_acc))
        print('Best val Pre: {:4f}'.format(best_precision))
        print('Best val Rec: {:4f}'.format(best_recall))
        print('Best val F1: {:4f}'.format(best_f1))
        print('Test Acc: {:4f}'.format(best_test_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
    return model

In [15]:
path = 'models/transfer_learning'
for seed in range(5):
    torch.manual_seed(seed)
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)

    model_ts = model_ts.to(device)

#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.]).to(device))
#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5.]).to(device))
    
    criterion = nn.BCEWithLogitsLoss()
    
    # Observe that all parameters are being optimized
    optimizer_ts = optim.SGD(model_ts.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler_ts = lr_scheduler.StepLR(optimizer_ts, step_size=7, gamma=0.1)
    
    if not os.path.exists(path):
        os.makedirs(path)
    with open(os.path.join(path, 'seed_{}.log'.format(seed)), 'w') as log_file:
        model_ts = train_model(model_ts, criterion, optimizer_ts, exp_lr_scheduler_ts,
                               'val_train', 'val', test_dl='test', num_epochs=25, verbose=True,
                               log_file=log_file, return_best=False)
        torch.save(model_ts.state_dict(), os.path.join(path, 'seed_{}.pth'.format(seed)))

val_train val test
Epoch 0/24
----------


C:\Users\mcram\Anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.3456 Acc: 0.8539 Pre: 0.7525 Rec: 0.7418 F1: 0.7471
TP: 2046.0 TN: 6048.0 FP: 673.0 FN: 712.0
val Loss: 0.1682 Acc: 0.9478 Pre: 0.9748 Rec: 0.8423 F1: 0.9037
TP: 2323.0 TN: 6661.0 FP: 60.0 FN: 435.0
test Loss: 0.2428 Acc: 0.9153 Pre: 0.9272 Rec: 0.7946 F1: 0.8558
TP: 1884.0 TN: 4977.0 FP: 148.0 FN: 487.0
Training time 15m 9s

Epoch 1/24
----------
train Loss: 0.2689 Acc: 0.8979 Pre: 0.8219 Rec: 0.8285 F1: 0.8252
TP: 2285.0 TN: 6226.0 FP: 495.0 FN: 473.0
val Loss: 0.0942 Acc: 0.9736 Pre: 0.9491 Rec: 0.9608 F1: 0.9550
TP: 2650.0 TN: 6579.0 FP: 142.0 FN: 108.0
test Loss: 0.2511 Acc: 0.9204 Pre: 0.8293 Rec: 0.9422 F1: 0.8821
TP: 2234.0 TN: 4665.0 FP: 460.0 FN: 137.0
Training time 29m 3s

Epoch 2/24
----------
train Loss: 0.2132 Acc: 0.9189 Pre: 0.8568 Rec: 0.8658 F1: 0.8613
TP: 2388.0 TN: 6322.0 FP: 399.0 FN: 370.0
val Loss: 0.1166 Acc: 0.9720 Pre: 0.9419 Rec: 0.9634 F1: 0.9525
TP: 2657.0 TN: 6557.0 FP: 164.0 FN: 101.0
test Loss: 0.2597 Acc: 0.9110 Pre: 0.8071 Rec: 0.9443 F1:

val Loss: 0.0461 Acc: 0.9854 Pre: 0.9733 Rec: 0.9768 F1: 0.9750
TP: 2694.0 TN: 6647.0 FP: 74.0 FN: 64.0
test Loss: 0.2325 Acc: 0.9317 Pre: 0.8582 Rec: 0.9393 F1: 0.8969
TP: 2227.0 TN: 4757.0 FP: 368.0 FN: 144.0
Training time 320m 34s

Epoch 23/24
----------
train Loss: 0.1261 Acc: 0.9535 Pre: 0.9166 Rec: 0.9242 F1: 0.9204
TP: 2549.0 TN: 6489.0 FP: 232.0 FN: 209.0
val Loss: 0.0440 Acc: 0.9876 Pre: 0.9800 Rec: 0.9772 F1: 0.9786
TP: 2695.0 TN: 6666.0 FP: 55.0 FN: 63.0
test Loss: 0.2206 Acc: 0.9357 Pre: 0.8708 Rec: 0.9355 F1: 0.9020
TP: 2218.0 TN: 4796.0 FP: 329.0 FN: 153.0
Training time 334m 25s

Epoch 24/24
----------
train Loss: 0.1264 Acc: 0.9536 Pre: 0.9136 Rec: 0.9282 F1: 0.9209
TP: 2560.0 TN: 6479.0 FP: 242.0 FN: 198.0
val Loss: 0.0451 Acc: 0.9868 Pre: 0.9786 Rec: 0.9761 F1: 0.9773
TP: 2692.0 TN: 6662.0 FP: 59.0 FN: 66.0
test Loss: 0.2197 Acc: 0.9342 Pre: 0.8665 Rec: 0.9363 F1: 0.9001
TP: 2220.0 TN: 4783.0 FP: 342.0 FN: 151.0
Training time 347m 59s


Training complete in 347m 59s
va

KeyboardInterrupt: 

# Smooth Results

In [17]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [18]:
path = 'models/transfer_learning'

log_file_img = open(os.path.join(
    path, 'test_results_image_classifier.log'), 'w')
log_file_smoothed = open(os.path.join(
    path, 'test_results_smoothed.log'), 'w')

for seed in range(5):
    print(seed)
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)
    model_ts.load_state_dict(torch.load(
        os.path.join(path, 'seed_{}.pth'.format(seed))
    ))
    model_ts.to(device)
    criterion = nn.BCEWithLogitsLoss()
    
    model = model_ts.eval()   # Set model to evaluate mode
    dl = dataloaders['test']
    dataset_size = dataset_sizes['test']
    
    running_corrects = 0
    true_positives = 0.
    true_negatives = 0.
    false_positives = 0.
    false_negatives = 0.

    i = 0

    predictions = []
    gt_labels = []
    # Iterate over data.
    for inputs, labels in dl:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            preds = torch.where(
                outputs >= 0.,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
            target = torch.where(
                labels >= 0.5,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device)
            )

        predictions += preds.cpu().numpy().tolist()
        gt_labels += labels.cpu().numpy().tolist()

        # statistics
        label_vals = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        correct = preds.view(label_vals.shape) == label_vals.data
        running_corrects += torch.sum(correct)

        true_positives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        true_negatives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_positives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_negatives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )

        num_fp = torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        )

    #     if num_fp > 0:
    #         print(num_fp)
    #         print(torch.where(
    #             (correct == 0.) * (label_vals == 0.),
    #             torch.tensor([1.]).to(device),
    #             torch.tensor([0.]).to(device)))
    #         print(i)
    #         out = torchvision.utils.make_grid(inputs)
    # #         imshow(out, title=preds.cpu().numpy().tolist())
    #         imshow(out)

        i += 1

    epoch_acc = running_corrects.double() / dataset_size
    epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
    epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
    epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

    print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
        epoch_acc, epoch_pre, epoch_recall, epoch_f1))
    print('TP: {} TN: {} FP: {} FN: {}'.format(
        true_positives.data, true_negatives.data, false_positives.data, 
        false_negatives.data))
    
    log_file_img.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                       f1=epoch_f1, tp=int(true_positives.data),
                       tn=int(true_negatives.data),
                       fp=int(false_positives.data), fn=int(false_negatives.data)
                   ))
    log_file_img.flush()

    predictions = [p[0] for p in predictions]

    smoothed_preds = smooth_predictions(predictions, 3)

    print("Smoothed stats:")
    print(acc_prf1(smoothed_preds, gt_labels))
    
    sm_acc, sm_pre, sm_rec, sm_f1, sm_tp, sm_tn, sm_fp, sm_fn = acc_prf1(
        smoothed_preds, gt_labels)
    
    log_file_smoothed.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=sm_acc, pre=sm_pre, rec=sm_rec,
                       f1=sm_f1, tp=sm_tp,
                       tn=sm_tn,
                       fp=sm_fp, fn=sm_fn
                   ))
    log_file_smoothed.flush()

0
Acc: 0.9400 Pre: 0.8912 Rec: 0.9228 F1: 0.9068
TP: 2188.0 TN: 4858.0 FP: 267.0 FN: 183.0
Smoothed stats:
(0.9340981856990395, 0.8813490451036164, 0.9148038802193167, 0.8977649006622517, 2169, 4833, 292, 202)
1
Acc: 0.9506 Pre: 0.9216 Rec: 0.9224 F1: 0.9220
TP: 2187.0 TN: 4939.0 FP: 186.0 FN: 184.0
Smoothed stats:
(0.9450373532550693, 0.9145154464663563, 0.9114297764656263, 0.912970004224757, 2161, 4923, 202, 210)
2
Acc: 0.9452 Pre: 0.8911 Rec: 0.9418 F1: 0.9157
TP: 2233.0 TN: 4852.0 FP: 273.0 FN: 138.0
Smoothed stats:
(0.9389007470651014, 0.8883475436459602, 0.9228173766343315, 0.9052544476623914, 2188, 4850, 275, 183)
3
Acc: 0.9344 Pre: 0.8609 Rec: 0.9452 F1: 0.9011
TP: 2241.0 TN: 4763.0 FP: 362.0 FN: 130.0
Smoothed stats:
(0.9288954108858057, 0.8601097178683386, 0.9257697174188106, 0.8917326833231769, 2195, 4768, 357, 176)
4
Acc: 0.9338 Pre: 0.8692 Rec: 0.9308 F1: 0.8990
TP: 2207.0 TN: 4793.0 FP: 332.0 FN: 164.0
Smoothed stats:
(0.9303628601921025, 0.8678869876641464, 0.91986503584

# Understand Mistakes

In [29]:
path = 'models/transfer_learning'
model_ts = models.resnet50(pretrained=True)
num_ftrs = model_ts.fc.in_features
model_ts.fc = nn.Linear(num_ftrs, 1)
model_ts.load_state_dict(torch.load(
    os.path.join(path, 'seed_0.pth')
))
model_ts.to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3.]).to(device))

In [30]:
model = model_ts.eval()   # Set model to evaluate mode
dl = dataloaders['test']
dataset_size = dataset_sizes['test']

In [31]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.cpu().numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(str(title))
    plt.pause(0.001)  # pause a bit so that plots are updated

In [32]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [33]:
running_corrects = 0
true_positives = 0.
true_negatives = 0.
false_positives = 0.
false_negatives = 0.

i = 0

predictions = []
gt_labels = []
# Iterate over data.
for inputs, labels in dl:
    inputs = inputs.to(device)
    labels = labels.to(device).float()

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        preds = torch.where(
            outputs >= 0.,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        target = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        
    predictions += preds.cpu().numpy().tolist()
    gt_labels += labels.cpu().numpy().tolist()

    # statistics
    label_vals = torch.where(
        labels >= 0.5,
        torch.tensor([1.]).to(device),
        torch.tensor([0.]).to(device)
    )
    correct = preds.view(label_vals.shape) == label_vals.data
    running_corrects += torch.sum(correct)

    true_positives += torch.sum(
        torch.where(
            (correct == 1.) * (label_vals == 1.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    true_negatives += torch.sum(
        torch.where(
            (correct == 1.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    false_positives += torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    false_negatives += torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 1.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    
    num_fp = torch.sum(
    torch.where(
        (correct == 0.) * (label_vals == 0.),
        torch.tensor([1.]).to(device),
        torch.tensor([0.]).to(device))
    )

#     if num_fp > 0:
#         print(num_fp)
#         print(torch.where(
#             (correct == 0.) * (label_vals == 0.),
#             torch.tensor([1.]).to(device),
#             torch.tensor([0.]).to(device)))
#         print(i)
#         out = torchvision.utils.make_grid(inputs)
# #         imshow(out, title=preds.cpu().numpy().tolist())
#         imshow(out)
    
    i += 1

epoch_acc = running_corrects.double() / dataset_size
epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
    epoch_acc, epoch_pre, epoch_recall, epoch_f1))
print('TP: {} TN: {} FP: {} FN: {}'.format(
    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))

predictions = [p[0] for p in predictions]

smoothed_preds = smooth_predictions(predictions, 3)

print("Smoothed stats:")
print(acc_prf1(smoothed_preds, gt_labels))

Acc: 0.9421 Pre: 0.8904 Rec: 0.9317 F1: 0.9106
TP: 2209.0 TN: 4853.0 FP: 272.0 FN: 162.0
Smoothed stats:
(0.9327641408751334, 0.8783948115119579, 0.9139603542808942, 0.8958247209590741, 2167, 4825, 300, 204)
